In [2]:
import pandas as pd
import numpy as np

# CellPhoneDB structure conversion

CellPhoneDB uses sql database, which is provided as .db, there are 6 tables, we extract these tables as csv. 

You can do it via sqlite tool, the below command extracts the gene_table and saves it as gene_table.csv

```sqlite3 -header -csv cellphone_pre.db "select * from gene_table;" > gene_table.csv```

Or you can use DB Browser for SQLite and, extract all the tables as csv through GUI. 


https://sqlitebrowser.org/




# Database structure 

Adapted from 

Efremova, M., Vento-Tormo, M., Teichmann, S. A., & Vento-Tormo, R. (2020). CellPhoneDB: inferring cell-cell communication from combined expression of multi-subunit ligand-receptor complexes. Nature protocols, 15(4), 1484–1506. https://doi.org/10.1038/s41596-020-0292-x

The database consists of 6 main tables: gene_table; protein_table; multidata_table;
interaction_table; complex_table; complex_composition_table

**gene_table**

This table stores all the information generated in the gene_input database input file. This
includes the gene name (“gene_name”); the HUGO nomenclature committee symbol (HGNC)
(“hgnc_symbol”) and the ensembl identifier (“ensembl”). Importantly, only the gene and protein
information of the interactions participants from “interactions_list” is stored in the database. 

The gene table is related to the protein table via the protein_id - id_protein (one to many)
foreign key.

**multidata_table**

This table stores the information shared between the protein_table and the complex_table.
All the information required in this table is obtained from the protein_input and complex_input
input files. It stores the following fields: (i) ‘name’, corresponding to uniprot if the specific entry (row)
represents a protein, or ‘complex_name’ if the entry represents a complex; (ii) ‘transmembrane’, (iii)
‘peripheral’, (iv) ‘secreted’, (v) ‘secreted_desc’, (vi) ‘secreted_highlight’, (vii) ‘receptor’, (viii) ‘receptor_desc’, (ix) ‘integrin’, (x) ‘other’ and (xi) ‘other_desc’. In addition, an ‘is_complex’ column is added
for internal optimization and indicates whether the entry (row) is a complex.


**protein_table**

This table stores the information obtained from the database input file, protein_input. It contains the
name of the protein (‘protein_name’), ‘tags’, ‘tags_reason’, ‘tags_description’ and ‘pfam’. The table is
related to multidata_Table (1..0 - 1 relation, meaning that one or zero elements of protein_table
correspond to one element of multidata_table) through the ‘protein_multidata_id’ foreign key.


**complex_table**


This table stores complex information from the database input file complex_input and stores the
following fields: ‘pdb_id’, ‘pdb_structure’, ‘stoichiometry’ and ‘comments_complex’. The table is
related to multidata_table (this is a 1..0 - 1 relation, meaning that one or zero elements of complex_table correspond to one element of multidata_table) through the ‘complex_multidata_id’
foreign key.
All information about the complex components is stored in the complex_composition_table file.

**complex_composition_table**


This table stores the proteins (‘uniprot_1’–‘uniprot_4’) that compose a complex. It is connected to
multidata_table through ‘complex_multidata_id’ and ‘protein_multidata_id’ (this is a 1..* - 1 relation,
meaning that multiple proteins and/or complexes with IDs stored in multidata_table can participate
in one complex_composition and can be included in the complex_composition_table). We also
created an additional column called ‘total_protein’ (with a number of complex components) for internal optimization purposes. 

**interaction_table**


This table stores the interaction data from the interaction_input file. The following columns are used
to represent the data: ‘id_cp_interaction’, ‘annotation_strategy’ and ‘source’. To identify the interaction partners (‘partner_a’ and ‘partner_b’ in interaction_input), the table is connected to multidata_table through the foreign keys ‘multidata_1_id’ and ‘multidata_2_id’, respectively, with a 1 - 1..*
relation, meaning that one multidata_id can be present multiple times in the interaction_table.
multidata_table stores both protein and complex data. Importantly, only genes and proteins participating in cell–cell communication are stored in our database; that is, not all the proteins present in
the input files are stored in our database 

In [1]:
#read extracted interaction table 
interaction = pd.read_csv ('interaction_table.csv', index_col=False)
interaction = interaction.iloc[:, [2,3]]

In [26]:
# complex_comp

In [27]:
multi = pd.read_csv ('multidata_table.csv', index_col=None)

In [28]:
# multi = pd.read_csv ('multidata_table.csv', index_col=None)

#we store the genes according to their function annotation. 
receptor = multi[multi['receptor'] == 1]
receptor = receptor.iloc[:, [0,1]]
ligand = multi[multi['receptor'] == 0]
ligand = ligand.iloc[:, [0,1]]

In [29]:
# We want to aggreate all the ligands under multidata_1_id column while receptors in multidata_2_id
# with this way, pairs will be always from Ligand to Receptor. Nothing can be done for the interactions
# between receptor-receptor. We keep it that way. 
r=interaction[interaction['multidata_1_id'].isin(receptor['id_multidata'])].copy()
#fix column order for l-r
r[['multidata_1_id', 'multidata_2_id']] = r[['multidata_2_id', 'multidata_1_id']]
l=interaction[interaction['multidata_1_id'].isin(ligand['id_multidata'])].copy()

In [30]:
i=pd.concat([l, r], ignore_index=True).copy()

In [14]:
# protein_name=gene_table

In [36]:
#read the tables.
gene_table = pd.read_csv ('gene_table.csv', index_col=False)
gene_table = gene_table.iloc[:, [3,4]].copy()
protein_name = pd.read_csv ('protein_table.csv', index_col=False)
protein_name = protein_name.iloc[:, [1,5]].copy()
complex_comp = pd.read_csv ('complex_composition_table.csv', index_col=False)
complex_comp = complex_comp.iloc[:, [1,2]].copy()

In [37]:
# we ll use gene_table to convert IDs to symbols
gene_table.columns=protein_name.columns
protein_name=gene_table

In [39]:
i["ligand"] = ''
i["receptor"] = ''
i["l_adhesion_mol"] = ''
i["r_adhesion_mol"] = ''
i["l_complex"] = ''
i["r_complex"] = ''

In [40]:
df = pd.DataFrame(columns=['L', 'R'])
df["ligand"] = ''
df["receptor"] = ''
df["l_adhesion_mol"] = ''
df["r_adhesion_mol"] = ''

In [41]:
# function to split complexes
def split_complex(multi_id):
    ids=complex_comp.loc[complex_comp['complex_multidata_id'] == multi_id].protein_multidata_id.values
    return list(ids)

In [42]:
#function to get the gene symbol from multidata_id
def get_name(prot_id):
    name=protein_name.loc[protein_name['protein_multidata_id'] == prot_id].protein_name
    name = name.str.replace('_HUMAN', '')
    return list(name)[0]

In [44]:
# the multidata_id for complex molecules are greater than 1282, so we deal with them seperately. 
for x in range(len(i)):
    entry={'L':[],'R':[]}
    p_id = i.loc[x].multidata_1_id #get the multidata_ids
    p_id2 = i.loc[x].multidata_2_id
    
    if p_id < 1282:
        if p_id in ligand.id_multidata.values: #if id is in ligand, append the value under L column (ligands)
            entry['L'].append(p_id)
        else:
            entry['R'].append(p_id) #else put it under R column, Receptors
    
    elif p_id > 1281:
        complex_ids = split_complex(p_id) #split the complex molecule ids. we use the above function
                                        #because complex molecules have different protein ids
        for perid in complex_ids:
            if perid in ligand.id_multidata.values:
                entry['L'].append(perid)
            else:
                entry['R'].append(perid)
    
    if p_id2 < 1281: #do the same thing for multidata_id_2
        if p_id2 in ligand.id_multidata.values:
            entry['L'].append(p_id2)
        else:
            entry['R'].append(p_id2)
        
    elif p_id2 > 1281:
        complex_ids = split_complex(p_id2)
        
        for perid in complex_ids:
            if perid in ligand.id_multidata.values:
                entry['L'].append(perid)
            else:
                entry['R'].append(perid)
                
    #for the complexes we produce all the possible pairs.             
    #for example we have a complex mol, L1_L2 interacting with R1_R2
    #we produce L1_R1, L1_R2 and L2_R1, L2_R2
    if entry.get('L') and entry.get('R'):
        data=((x,y) for x in entry.get('L') for y in entry.get('R'))
        data=pd.DataFrame(data,columns=['L', 'R'])
        data['pid1']=p_id.astype('int')
        data['pid2']=p_id2.astype('int')
        data['l_adhesion_mol']=0
        data['r_adhesion_mol']=0
        df=pd.concat([df, data], ignore_index=True)
    else:
        if entry.get('L'):
            data=((x,y) for x in entry.get('L') for y in entry.get('L'))
            data=pd.DataFrame(data,columns=['L', 'R'])
            data['pid1']=p_id.astype('int')
            data['pid2']=p_id2.astype('int')
            data['l_adhesion_mol']=1
            data['r_adhesion_mol']=0
            df=pd.concat([df, data], ignore_index=True)
        else:
            data=((x,y) for x in entry.get('R') for y in entry.get('R'))
            data=pd.DataFrame(data,columns=['L', 'R'])
            data['pid1']=p_id.astype('int')
            data['pid2']=p_id2.astype('int')
            data['r_adhesion_mol']=1
            data['l_adhesion_mol']=0
            df=pd.concat([df, data], ignore_index=True)

In [45]:
for x in range(len(df)):
    l = df.loc[x].L
    r = df.loc[x].R
    l_name=get_name(l)
    r_name=get_name(r)
    df.at[x, 'ligand'] = l_name
    df.at[x, 'receptor'] = r_name

In [46]:
df

,L,R,ligand,receptor,l_adhesion_mol,r_adhesion_mol,pid1,pid2
0,901,917,PTGR1,LTB4R,0,0,1282.0,917.0
1,315,917,LTA4H,LTB4R,0,0,1430.0,917.0
2,901,1189,PTGR1,LTB4R2,0,0,1282.0,1189.0
3,315,1189,LTA4H,LTB4R2,0,0,1430.0,1189.0
4,315,864,LTA4H,GPR17,0,0,1430.0,864.0
...,...,...,...,...,...,...,...,...
4503,1177,418,CRLF2,IL7R,0,1,1500.0,1177.0
4504,1177,1177,CRLF2,CRLF2,0,1,1500.0,1177.0
4505,1177,1177,CRLF2,CRLF2,0,1,1500.0,1177.0
4506,1109,1090,NELL2,ROBO3,0,0,1109.0,1090.0


In [47]:
df.drop(df[df.ligand == df.receptor].index, inplace=True)

In [48]:
df = df.reset_index(drop=True)

In [49]:
df["pairs"]=df[["ligand", "receptor"]].apply("_".join, axis=1)

In [50]:
df[df.duplicated('pairs')]

,L,R,ligand,receptor,l_adhesion_mol,r_adhesion_mol,pid1,pid2,pairs
47,138,1252,WNT11,FZD10,0,0,1358.0,138.0,WNT11_FZD10
48,138,102,WNT11,LRP5,0,0,1359.0,138.0,WNT11_LRP5
50,138,108,WNT11,LRP6,0,0,1360.0,138.0,WNT11_LRP6
51,138,1260,WNT11,FZD1,0,0,1360.0,138.0,WNT11_FZD1
52,138,102,WNT11,LRP5,0,0,1361.0,138.0,WNT11_LRP5
...,...,...,...,...,...,...,...,...,...
3762,708,556,RXRG,CRABP2,1,0,1513.0,1486.0,RXRG_CRABP2
3763,556,698,CRABP2,ALDH1A3,1,0,1513.0,1486.0,CRABP2_ALDH1A3
3764,556,708,CRABP2,RXRG,1,0,1513.0,1486.0,CRABP2_RXRG
3766,418,1177,IL7R,CRLF2,0,1,1500.0,1177.0,IL7R_CRLF2


In [51]:
df=df.drop_duplicates('pairs')

In [52]:
df.to_csv('cpdb_revised2.csv2', index=False)

In [53]:
df['receptor'].isnull().values.any()

False

In [ ]:
cpdbnodes=list(df["ligand"])+list(df["receptor"])